# Notebook de versement de données

Ce notebook permet de verser des données Excel dans une base PostgreSQL.

## 1. Importation des packages

In [2]:
import sys
sys.path.append('..')

from script.body.base_donnees import ConnectionBaseDeDonnees
from script.body.livre import Livre
from script.body.traitement import Traitement
from script.body.versement import Versement
from script.branch.gestion_dossier import GestionDossier
from script.leaf.validator import DataValidator
from script.leaf.catalogue import Catalogue
from script.branch.gomme import Gomme

## 2. Configuration de la connexion

In [2]:
# Connexion à la base de données
db = ConnectionBaseDeDonnees()
conn, schema = db.connexion_observatoire()

Connexion réussie à Recette test !


## 3. Configuration du versement

In [3]:
# Paramètres du versement
fichier_excel = "../data/tuto.xlsx"
annee = 2025
theme = "contrainteag2"
base = "table"
source = "valeur"

# Validation du fichier
if not DataValidator.validate_excel_file(fichier_excel):
    raise ValueError("Fichier Excel invalide.")
else:
    print("Fichier Excel valide.")

# Enregistrement des paramètres du versement
livre = Livre(
    conn, 
    schema=schema, 
    theme=theme,
    base=base, 
    source=source, 
    annee=annee
)
livre.echelle = livre.choix_echelle()

Vous avez choisi l'échelle commune.


## Catalogue / Facultatif

In [ ]:
catalogue = Catalogue(livre)
catalogue.liste_table(afficher=True)

In [ ]:
catalogue.catalogue()

## 4. Traitement des données

In [4]:
# Traitement des données
traiteur = Traitement(conn, fichier_excel, livre)
traiteur.traitement()

Données lues.
Données sérialisées.
ID ajoutés.
Fichiers vars temporaires créés. Son nom est : traitement_20250218160627
Dossier temporaire créé.
contrainteag2_test_var.csv a été créé dans le dossier traitement_20250218160627.
contrainteag2_test_mod.csv a été créé dans le dossier traitement_20250218160627.
contrainteag2_table_valeur.csv a été créé dans le dossier traitement_20250218160627.
contrainteag2_test_vers.csv a été créé dans le dossier traitement_20250218160627.
Données enregistrées.
traitement a pris 0.68 secondes


## 5. Sauvegarde en base de données

In [5]:
# Sauvegarde des données
versement = Versement(conn, livre)
versement.versement()

CREATE TABLE test.contrainteag2_test_mod (id_mod INTEGER, nom_mod VARCHAR(8000), joli_nom_mod VARCHAR(8000), mod_regroupement VARCHAR(8000), lib_long_mod VARCHAR(8000));
Table contrainteag2_test_mod créée.
Données de contrainteag2_test_mod insérées.
CREATE TABLE test.contrainteag2_test_var (id_var INTEGER, nom_var VARCHAR(8000), joli_nom_var VARCHAR(8000), var_regroupement VARCHAR(8000), lib_long_var VARCHAR(8000));
Table contrainteag2_test_var créée.
Données de contrainteag2_test_var insérées.
CREATE TABLE test.contrainteag2_test_vers (id_versement INTEGER, nom_table VARCHAR(8000), annee INTEGER, echelle VARCHAR(8000), theme VARCHAR(8000), source VARCHAR(8000), commentaire VARCHAR(8000), url VARCHAR(8000));
Table contrainteag2_test_vers créée.
Données de contrainteag2_test_vers insérées.
CREATE TABLE test.contrainteag2_table_valeur (id_composite VARCHAR(8000), id_versement INTEGER, annee INTEGER, echelle VARCHAR(8000), code_entite INTEGER, id_var INTEGER, id_mod INTEGER, valeur INTEGE

## 6. Suppression des fichiers temporaires

In [11]:
gestionnaire_de_dossier = GestionDossier(livre)
gestionnaire_de_dossier.delete_file()

Le dossier traitement_20250218095201 a été supprimé.


## 7. Suppression de table / versement

In [ ]:
gomme = Gomme(conn, livre)
gomme.suppression_table(table_cible='', id_versement_cible=None, garder_table=False)

## 8. Ajout des contraintes sur les tables en production

In [3]:
# Connexion à la base de données
db = ConnectionBaseDeDonnees()
conn, schema = db.connexion_observatoire()

theme = "contrainte"
base = "table"
source = "valeur"
annee=2025

livre = Livre(
    conn, 
    schema=schema, 
    theme=theme,
    base=base, 
    source=source, 
    annee=annee
)

Connexion réussie à Recette ODH !


In [4]:
livre.relations = {'primaire': {'contrainte_table_valeur': 'id_composite',
  'contrainte_test_mod': 'id_mod',
  'contrainte_test_var': 'id_var',
  'contrainte_test_vers': 'id_versement'},
 'etrangere': {'contrainte_test_mod': ('id_mod', 'id_mod'),
  'contrainte_test_var': ('id_var', 'id_var'),
  'contrainte_test_vers': ('id_versement', 'id_versement')}}

In [5]:
versement = Versement(conn, livre)
tables = [
    'contrainte_table_valeur', # livre.nom_table,
    'contrainte_test_var', # livre.nom_table_var,
    'contrainte_test_mod', # livre.nom_table_mod,
    'contrainte_test_vers', # livre.nom_table_vers,
]
print(schema)
for nom in tables:
    print(nom)
versement.ajout_contraintes(schema, tables)

hab
contrainte_table_valeur
contrainte_test_var
contrainte_test_mod
contrainte_test_vers
Contrainte primaire de contrainte_table_valeur ajoutée.
Contrainte primaire de contrainte_test_var ajoutée.
Contrainte primaire de contrainte_test_mod ajoutée.
Contrainte primaire de contrainte_test_vers ajoutée.
cles_etrangeres:{'contrainte_test_mod': ('id_mod', 'id_mod'), 'contrainte_test_var': ('id_var', 'id_var'), 'contrainte_test_vers': ('id_versement', 'id_versement')}
Contraintes secondaires de contrainte_table_valeur ajoutées.
ajout_contraintes a pris 0.07 secondes
